## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
* torch.nn.AveragePool2d(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* torch.nn.AdaptiveAvgPool2d(output_size)
* torch.nn.AdaptiveMaxPool2d(output_size)

Layer de liniarizare:

* torch.nn.Flatten()



In [1]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(2,2), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 50, 50])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 24, 24]
*   [1, 10, 9, 9]
*  [1, 3, 2, 2]



In [1]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale
layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(8,8), stride=(4,4))
print("Conv2 result shape",layer1(dummy_input_tensor).shape)

layer2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(11,11), stride=(10,10))  
# kernel size de la 11 pana la 20 da acelasi rezultat
print("Conv2 result shape",layer2(dummy_input_tensor).shape)

layer3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(10,10), stride=(50,50))
print("Conv2 result shape",layer3(dummy_input_tensor).shape)


layer4 = nn.MaxPool2d(kernel_size=(35,35)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer4(dummy_input_tensor).shape)



layer5 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(1,1))
print("Conv2 result shape",layer5(dummy_input_tensor).shape)
a = layer5(dummy_input_tensor)
layer6 = nn.MaxPool2d(kernel_size=(4,4))
print("MaxPool result shape", layer6(a).shape)

Conv2 result shape torch.Size([1, 10, 24, 24])
Conv2 result shape torch.Size([1, 10, 9, 9])
Conv2 result shape torch.Size([1, 3, 2, 2])
Pool result shape torch.Size([1, 3, 2, 2])
Conv2 result shape torch.Size([1, 10, 98, 98])
MaxPool result shape torch.Size([1, 10, 24, 24])


## Instantierea seturilor de date

In [2]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)


Extracting ./data/cifar-10-python.tar.gz to ./data


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __preproc_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *preproc_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def preproc_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    normalized_tensor_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(normalized_tensor_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)
  
  return torch_images, torch_labels

batch_size=500
train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=preproc_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=preproc_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori
    * Liniarizati iesirea din cel de-al doilea strat convolutional
    * Adaugat stratul final de tipul 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [4]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
    self.layer2 = nn.Conv2d(in_channels=10, out_channels=5, kernel_size=(3,3), stride=(2,2))
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(245,10) #noi avem 32x32x3, deci (32-2):2 aka (32-kernel_size-1)/stride
    self.activation = nn.ReLU() 
  def forward(self,x):
    output_layer1 = self.layer1(x)
    output1 = self.activation(output_layer1)
    output_layer2 = self.layer2(output1)
    output2 = self.flatten(output_layer2)
    output2 = self.activation(output2)
    output_linear = self.linear(output2)
    output = self.activation(output_linear)
    return output

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci
  * Retea
  * Optimizator
  * Alegeti functia de cost

In [5]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 5

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(), lr=1e-2,momentum=0.8)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [12]:
def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  use_cuda = True
  # Iteram prin numarul de epoci
  for e in range(epochs):
    # Iteram prin fiecare exemplu din dataset
    for images, labels in train_loader:
      if use_cuda:
        images = images.cuda()
        labels = labels.cuda()
      # Aplicam reteaua neurala pe imaginile de intrare
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe adnotarile imaginilor 
      loss = loss_fn(out, labels.cuda())
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a aplica gradientii pe parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Caluculul acuratetii
    count = len(test_loader)
    correct = 0

    for test_image, test_label in test_loader:
      if use_cuda:
        test_label = test_label.cuda()
        test_image = test_image.cuda()
      out_class = torch.argmax(net(test_image))
      if out_class == test_label:
        correct += 1

    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e, (correct / count) * 100))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [14]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.281018018722534
Acuratetea la finalul epocii 0 este 18.91%
Loss-ul la finalul epocii 1 are valoarea 2.10899019241333
Acuratetea la finalul epocii 1 este 26.83%
Loss-ul la finalul epocii 2 are valoarea 1.9702876806259155
Acuratetea la finalul epocii 2 este 32.41%
Loss-ul la finalul epocii 3 are valoarea 1.903746485710144
Acuratetea la finalul epocii 3 este 33.76%
Loss-ul la finalul epocii 4 are valoarea 1.930052399635315
Acuratetea la finalul epocii 4 este 35.55%


## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet


In [15]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.layer1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=(1,1))
    self.activation = nn.ReLU()# nn.GELU()
    self.avgPool1 = nn.AvgPool2d(kernel_size = (2,2), stride=(2,2), padding=0)
    self.layer2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1))
    # self.avgPool2 = nn.AveragePool2d(kernel_size = (2,2), stride=(2,2), padding=0)
    self.layer3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1))
    self.linear = nn.Linear(120,84)
    self.linear2 = nn.Linear(84,10)
    self.softmax = nn.Softmax(dim = 1)
    self.flatten = nn.Flatten()
    #self.norm1 = nn.BatchNorm2d(6)

  def forward(self,x):
    output_layer1 = self.layer1(x)
    output1 = self.activation(output_layer1)
    output_avgPool1 = self.avgPool1(output1)
    output2 = self.activation(output_avgPool1)
    #output2 = self.norm1(output2)
    output_layer2 = self.layer2(output2)
    output3 = self.activation(output_layer2)
    output_avgPool1 = self.avgPool1(output3)
    output4 = self.activation(output_avgPool1)
    
    output_layer3 = self.layer3(output4)
    output5 = self.activation(output_layer3)
    output5 = self.flatten(output5)
    output_linear = self.linear(output5)
    output6 = self.activation(output_linear)
    output_linear2 = self.linear2(output6)
    output7 = self.softmax(output_linear2)
    


    return output7

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [20]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 8

# Definiti reteaua
lenet = LeNet().cuda()

# Definiti optimizatorul
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [43]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer) #norm1 la output2 config 3 GELU DEZAMAGIRE

Loss-ul la finalul epocii 0 are valoarea 2.1009340286254883
Acuratetea la finalul epocii 0 este 23.70%
Loss-ul la finalul epocii 1 are valoarea 2.1872806549072266
Acuratetea la finalul epocii 1 este 22.41%
Loss-ul la finalul epocii 2 are valoarea 2.2541580200195312
Acuratetea la finalul epocii 2 este 17.74%
Loss-ul la finalul epocii 3 are valoarea 2.2389471530914307
Acuratetea la finalul epocii 3 este 16.00%
Loss-ul la finalul epocii 4 are valoarea 2.267254114151001
Acuratetea la finalul epocii 4 este 14.85%
Loss-ul la finalul epocii 5 are valoarea 2.3111379146575928
Acuratetea la finalul epocii 5 este 12.56%
Loss-ul la finalul epocii 6 are valoarea 2.265991449356079
Acuratetea la finalul epocii 6 este 15.64%
Loss-ul la finalul epocii 7 are valoarea 2.243258476257324
Acuratetea la finalul epocii 7 este 15.28%


In [40]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer) #regularizare norm1 la output4 bad config 2 GELU

Loss-ul la finalul epocii 0 are valoarea 2.0545542240142822
Acuratetea la finalul epocii 0 este 27.14%
Loss-ul la finalul epocii 1 are valoarea 2.100139856338501
Acuratetea la finalul epocii 1 este 24.59%
Loss-ul la finalul epocii 2 are valoarea 2.156421661376953
Acuratetea la finalul epocii 2 este 24.60%
Loss-ul la finalul epocii 3 are valoarea 2.205085515975952
Acuratetea la finalul epocii 3 este 21.52%
Loss-ul la finalul epocii 4 are valoarea 2.16874623298645
Acuratetea la finalul epocii 4 este 24.57%
Loss-ul la finalul epocii 5 are valoarea 2.1809921264648438
Acuratetea la finalul epocii 5 este 19.94%
Loss-ul la finalul epocii 6 are valoarea 2.1723978519439697
Acuratetea la finalul epocii 6 este 21.19%
Loss-ul la finalul epocii 7 are valoarea 2.2050604820251465
Acuratetea la finalul epocii 7 este 18.84%


In [19]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)  #initial config (tanh activation function)

Loss-ul la finalul epocii 0 are valoarea 2.1028990745544434
Acuratetea la finalul epocii 0 este 33.39%
Loss-ul la finalul epocii 1 are valoarea 2.0748846530914307
Acuratetea la finalul epocii 1 este 37.36%
Loss-ul la finalul epocii 2 are valoarea 2.0458686351776123
Acuratetea la finalul epocii 2 este 39.49%
Loss-ul la finalul epocii 3 are valoarea 2.0380918979644775
Acuratetea la finalul epocii 3 este 42.25%
Loss-ul la finalul epocii 4 are valoarea 2.014744758605957
Acuratetea la finalul epocii 4 este 43.87%
Loss-ul la finalul epocii 5 are valoarea 2.0127031803131104
Acuratetea la finalul epocii 5 este 44.88%
Loss-ul la finalul epocii 6 are valoarea 1.9859122037887573
Acuratetea la finalul epocii 6 este 45.78%
Loss-ul la finalul epocii 7 are valoarea 1.9943145513534546
Acuratetea la finalul epocii 7 este 46.98%


In [21]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer) ##initial config (tanh activation function) config 2 RELU

Loss-ul la finalul epocii 0 are valoarea 2.112945318222046
Acuratetea la finalul epocii 0 este 34.58%
Loss-ul la finalul epocii 1 are valoarea 2.0848426818847656
Acuratetea la finalul epocii 1 este 37.67%
Loss-ul la finalul epocii 2 are valoarea 2.066033363342285
Acuratetea la finalul epocii 2 este 40.18%
Loss-ul la finalul epocii 3 are valoarea 2.0117897987365723
Acuratetea la finalul epocii 3 este 42.14%
Loss-ul la finalul epocii 4 are valoarea 2.0250701904296875
Acuratetea la finalul epocii 4 este 43.86%
Loss-ul la finalul epocii 5 are valoarea 2.0057997703552246
Acuratetea la finalul epocii 5 este 45.78%
Loss-ul la finalul epocii 6 are valoarea 1.9796842336654663
Acuratetea la finalul epocii 6 este 47.02%
Loss-ul la finalul epocii 7 are valoarea 1.967206597328186
Acuratetea la finalul epocii 7 este 46.62%


###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:


* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
